In [2]:
//Model as an ADT
object model {
  case class ProcessComponent(desc: String)

  type Process = Seq[ProcessComponent]
  val emptyProcess = Seq.empty[ProcessComponent]
  
  
  sealed trait TicketStatus
  case object Open extends TicketStatus
  case object InProgress extends TicketStatus
  case object Closed extends TicketStatus
  
  //Aggregate root
  case class Ticket(no:String, status: TicketStatus, desc: String, process: Process)
}

defined object model

Simple domain service

In [3]:
import model._
//Let's define a service that describes our algebra
object services {
  trait TicketService[Ticket, TicketStatus, Process] {
    def open(no: String, desc: String, process: Process): Ticket
    def changeStatus(no: String, status: TicketStatus): Ticket 
    def changeDescription(no: String, descr: String): Ticket 
    def close(no: String): Ticket
  }
  
  //Concrete implementation
  object TicketService extends TicketService[Ticket, TicketStatus, Process] {
  
    def open(no: String, desc: String, process: Process): Ticket 
      = Ticket(no, Open, desc, process)
      
    def changeStatus(no: String, status: TicketStatus): Ticket = ???
   
    def changeDescription(no: String, descr: String): Ticket = ???
   
    def close(no: String): Ticket = ???
  }
  
}



//Sample of use
val TS = services.TicketService
TS.open("t1", "First ticket", model.emptyProcess )
TS.open("t2", "Second ticket", model.emptyProcess )
TS.open("t3", "Second ticket", model.emptyProcess )

import model._
defined object services
TS: $user.services.TicketService.type = cmd2$$user$services$TicketService$@6abc2e5c
res2_3: model.Ticket = Ticket(t1,Open,First ticket,List())
res2_4: model.Ticket = Ticket(t2,Open,Second ticket,List())
res2_5: model.Ticket = Ticket(t3,Open,Second ticket,List())

Define a domain repository 
* Repository may fail
* Tickets may not exist

In [5]:
import scala.util.{Try, Success, Failure}
import collection.mutable.{ Map => MMap }


object repos {

  trait TicketRepository {
    def query(): Try[Seq[Ticket]]
    def query(no:String): Try[Option[Ticket]]
    def store(t: Ticket): Try[Ticket]
  }
  
  
  
  trait InMemoryTicketRepository extends TicketRepository {
    lazy val repo = MMap.empty[String, Ticket]
    def query(): Try[Seq[Ticket]] = Success(repo.values.toSeq)
    
    def query(no: String): Try[Option[Ticket]] = Success(repo.get(no))
    
    def store(a: Ticket): Try[Ticket] = {
      val r = repo += ((a.no, a))
      Success(a)
    }
    
  }
  
  object InMemoryTicketRepository extends InMemoryTicketRepository
}

import scala.util.{Try, Success, Failure}
import collection.mutable.{ Map => MMap }
defined object repos

Inject the repo to our ticket service the functional way
* This is another implementation of dependency injection

In [10]:
//Let's define a service that describes our algebra
object services {
  import repos.TicketRepository
  
  trait TicketService[Ticket, TicketStatus, Process] {
    def open(no: String, desc: String, process: Process): TicketRepository => Try[Ticket]
//     def changeStatus(no: String, status: TicketStatus): TicketRepository => Try[Ticket] 
//     def changeDescription(no: String, descr: String): TicketRepository => Try[Ticket] 
//     def close(no: String): Try[Ticket]
  }
  
  //Concrete implementation
  object TicketService extends TicketService[Ticket, TicketStatus, Process] {
  
    def open(no: String, desc: String, process: Process) = { (r: TicketRepository) =>
      Success(Ticket(no, Open, desc, process))
      
    } 
      
      
//     def changeStatus(no: String, status: TicketStatus): Ticket = ???
   
//     def changeDescription(no: String, descr: String): Ticket = ???
   
//     def close(no: String): Ticket = ???
  }
  
}


val memoryRepo =  repos.InMemoryTicketRepository 

val TS = services.TicketService
TS.open("t1", "First ticket", model.emptyProcess)(memoryRepo)




memoryRepo.repo



defined object services
memoryRepo: repos.InMemoryTicketRepository.type = cmd4$$user$repos$InMemoryTicketRepository$@679cfd4f
TS: $user.services.TicketService.type = cmd9$$user$services$TicketService$@48e44b7b
res9_3: Success[Ticket] = Success(Ticket(t1,Open,First ticket,List()))
res9_4: collection.mutable.Map[String, Ticket] = Map()